In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

# Список республик бывшего СССР
resp = ['Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania',\
        'Republic of Moldova', 'Russian Federation', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan']

Для начала посмотрим статистику по "Ожидаемая продолжительность жизни". Возьмем информацию только для стран бывшего СССР, только за 2019 и общую для обеих полов.

In [2]:
# Фунция для стандартного выбора данных из таблиц
# Передаем в нее название файла, имена столбцов, название нужной нам переменной, период и условие
def get_stat(filename, nms, val, period, condition1='', condition2=''):
    url = "./dataframes/" + filename + ".csv"
    data = pd.read_csv(url, dtype = {'country': str, 'period': int},\
                       header=None, skiprows=[0,], names=nms)
    data = data[(data['country'].isin(resp)) & (data['period']==period)]
    if(condition1!='' and condition2!=''):
        data = data[(data[condition1]==condition2)]
    return data[['country', val]].sort_values(by=val, ascending=False)

#pd.pivot_table(data, values = 'age', columns = 'country')

datas = {} # В этот словарь будем собирать данные по странам

In [3]:
datas['leab'] = get_stat('lifeExpectancyAtBirth',
                     ['country','period','indicator','gender','age'],                         
                     'age',
                     2019,
                     'gender', 'Both sexes')
datas['leab']

,country,age
649,Estonia,78.88
72,Armenia,76.03
1153,Lithuania,75.99
1093,Latvia,75.38
168,Belarus,74.81
1069,Kyrgyzstan,74.18
1021,Kazakhstan,73.95
745,Georgia,73.28
1585,Republic of Moldova,73.26
1621,Russian Federation,73.23


Как можно увидеть по таблице выше на первых местах по продолжительности жизни расположились Прибалтийские страны (Эстония, Латвия, Литва) и Армения. На последних местах Азербайджан, Туркменистан и Таджикистан. Россия в списке находится на 10 месте.
<br><br>
Попробуем понять, что на это влияет. Для начала рассмотрим уровень медицины в этих странах и насколько он различается.
1. Сравним какое кол-во докторов приходится на каждые  на 10 000 населения.
<br>
(т.к. данные по всем странам бывшего СССР есть только за 2014 год, то для сравнения возьмем данные именно за этот год)

In [4]:
datas['md'] = get_stat('medicalDoctors',
                     ['country','period','indicator','doctors'],                       
                     'doctors',
                     2014)
datas['md']

,country,doctors
206,Belarus,50.12
844,Georgia,47.75
1316,Lithuania,42.51
1871,Russian Federation,40.11
1176,Kazakhstan,39.80
138,Azerbaijan,34.46
726,Estonia,33.56
1262,Latvia,31.72
2344,Ukraine,29.92
70,Armenia,28.93


Как можно увидеть из таблицы выше, кол-во врачей прямо не коррелируются с продолжительностью жизни. Единственное совпадение это Туркменистан и Таджикистан с наименьшим кол-вом врачей на 10 т.ч. и с наименьшой продолжительностю жизни среди других стран.
<br>
2. Теперь посмотрим как обстоит дело с медсестринским и акушерским персоналом на 10 000 населения.

In [5]:
datas['nam'] = get_stat('nursingAndMidwife',
                     ['country','period','indicator','nursing'],                        
                     'nursing',
                     2014)
datas['nam']

,country,nursing
2510,Uzbekistan,112.80
200,Belarus,109.80
1361,Lithuania,78.05
1205,Kazakhstan,72.00
2435,Ukraine,66.61
134,Azerbaijan,64.34
772,Estonia,60.72
1263,Kyrgyzstan,59.45
1884,Republic of Moldova,53.11
75,Armenia,51.55


Здесь также нельзя провести прямую связь между продолжительностью жизни и кол-вом мед.персонала.
<br>
Попробуем предположить, что на это влияют различные факторы в стране, такие как чистая питьевая вода, санитария и гигиена.
<br>
3. Посмотрим какой процент населения обеспечены питьевой водой

In [6]:
datas['bdws'] = get_stat('basicDrinkingWaterServices',
                     ['country','period','indicator','percent'],                         
                     'percent',
                     2017)
datas['bdws']

,country,percent
126,Armenia,100.00
1166,Georgia,100.00
3168,Turkmenistan,100.00
1022,Estonia,99.19
1688,Latvia,98.18
3329,Uzbekistan,96.07
270,Belarus,95.98
1778,Lithuania,92.76
2555,Russian Federation,92.62
1580,Kazakhstan,92.31


На первых 3 местах видим Армению, Грузию и Туркменистан со 100% обеспечения населения питьевой водой. Эти цифры вызвают сомнения, т.к. в данных странах присутствуют удаленные населенные пункты, которые сложно обеспечить питьевой водой.
<br>
Также обратимся к отчету Европейского Агентства по окружающей среде, согласно ему:
<br>
<i>"В 2017 году почти 98 % сельского и 100 % городского населения Грузии имели доступ к питьевому водоснабжению, отвечающему требованиям безопасности"</i>
<br>
https://www.eea.europa.eu/ru/publications/vodnye-resursy-kachestvo-poverhnostnyh-vod/
<br><br>
Как можно видеть эти цифры отличаются от того, что мы получили из выборке за 2017 год. Поэтому рассмотрим даные про обеспечение водой за предыдущий год, т.е. за 2016.

In [7]:
datas['bdws'] = get_stat('basicDrinkingWaterServices',
                     ['country','period','indicator','percent'],                         
                     'percent',
                     2016)
datas['bdws']

,country,percent
127,Armenia,100.00
1023,Estonia,99.19
1689,Latvia,98.03
3169,Turkmenistan,97.72
271,Belarus,95.96
3330,Uzbekistan,95.86
1167,Georgia,95.71
2556,Russian Federation,92.14
1779,Lithuania,91.74
3222,Ukraine,91.32


Здесь данные похожи на достоверные и можно провести некую связь между людьми с доступом к питьевой воде и продолжительностю жизни.
<br>
На первых 3 местах мы можем видеть Армению, Эстонию и Латвию. На 2-х последних местах мы видим все те же Азербайджан и Таджикистан.
<br><br>
4. Теперь обратимся к чистым видам топлива и технологиям:

In [8]:
datas['cfat'] = get_stat('cleanFuelAndTech',
                     ['country','indicator','period','percent'],
                     'percent',
                     2018)
datas['cfat']

,country,percent
133,Armenia,95
190,Azerbaijan,95
285,Belarus,95
1045,Estonia,95
1634,Kazakhstan,95
1748,Latvia,95
1805,Lithuania,95
2584,Republic of Moldova,95
3306,Turkmenistan,95
3363,Ukraine,95


Здесь мы можем видеть, что 2/3 стран имеют одинаковый процент населения, которое полагается на чистые виды топлива и технологии, т.е. 95%. Если посмотреть данные по этим странам за предыдущие годы, то у них также будет указано 95%. В данном случае мы предполагаем, что это шаблонные данные, которые без изменений добавляли в эту таблицу из года в год и поэтому мы не можем их использовать для дальнейших расчетов.
<br><br>
5. Рассмотрим какой процент населения имеет доступ к базовым санитарным услугам

In [9]:
datas['albss'] = get_stat('atLeastBasicSanitizationServices',
                     ['country','indicator','period','type','percent'],                          
                     'percent',
                     2017,
                     'type', 'Total')
datas['albss']

,country,percent
9031,Uzbekistan,100.00
2765,Estonia,99.15
8584,Turkmenistan,98.70
4349,Kazakhstan,97.87
628,Belarus,97.79
8194,Tajikistan,97.02
4493,Kyrgyzstan,96.51
8743,Ukraine,96.22
340,Armenia,93.64
4799,Lithuania,93.35


Как можно увидеть, то население всех стран было примерно в равной степени обесечено базовыми санитарными услугами. В данной выборке мы видм явного аутсайдера это Молдавию. Такая разница в данных может говорить или о разнице подсчетов данного параметра в странах или о действительно более плохом положении с санитарными услугами в Молдавии.
<br><br>
Обратимся снова к докладу Европейской коммиссии по окружающей среде и вот, что мы там видим:
<br>
<i>В 2017 году около 17 % сельского населения Молдовы (т.е. 7 % населения страны) не имели доступа к услугам водоснабжения, отвечающим требованиям безопасности, или услугам базового уровня. 31 % сельского населения имел ограниченный доступ к услугам водоотведения и канализации или доступ только к базовой санитарно-гигиенической инфраструктуре (WHO/UNICEF, 2019).</i>
<br>
Мы можем предположить, что эти цифры по Молдавии верны.
<br><br>
6. Теперь обратимся к статистике по обеспечению всеобщего охвата услугами здравоохранения 

In [10]:
datas['uc'] = get_stat('uhcCoverage',
                     ['country','indicator','period','percent'],                       
                     'percent',
                     2017)
datas['uc']

,country,percent
28,Belarus,76
170,Kazakhstan,76
108,Estonia,75
270,Russian Federation,75
192,Lithuania,73
352,Uzbekistan,73
182,Latvia,71
178,Kyrgyzstan,70
336,Turkmenistan,70
12,Armenia,69


Здесь мы можем увидеть, что на одном из превых 3 мест находится только Эстония. А среди 3 последних находим Азербайджан. Остальные страны распределились достаточно равномерно по таблице. Единственное, что все же республики с более высокой продолжительностю жизни находятся в верхней половине таблицы, а с низкой продолжительностю находятся в нижней. Только одно исключение это Армения.
<br>
На основании этих данных мы можем предположить, что обеспечение всеобщего охвата услугами здравоохраения косвернно влияет на продолжительсть жизни.
<br><br>
Теперь рассмотрим насколько загрязнение окружающей среды влияет на уроверь смертности.
<br>
7. Рассмотрим уровень смертности от загрязнений атмосферного и домашнего воздуха на 100 000 человек
<br>
(возьмем полную статистику про оба пола, 2016 год и со стандартизацией по возрасту)

In [121]:
# В данном случае получилось много условий и поэтому мы не стали применять стандарную функцию

# Т.к. в строке с уровнем смертности есть еще дополнительная информация через пробел, то уберем ее
f = lambda x : (x.split(" ")[0])

url = "./dataframes/airPollutionDeathRate.csv"
data = pd.read_csv(url, usecols=['country','type','indicator','period','gender','percent'], \
                        dtype = {'country': str, 'period': int, 'gender': str},\
                        converters = {'percent':f},\
                        header=None, skiprows=[0,], names=['country','type','indicator','period','gender','percent'])

data = data[(data['country'].isin(resp)) &\
            (data['type']=='Total') &\
            (data['gender']=='Both sexes') &\
            (data['indicator']=='Ambient and household air pollution attributable death rate (per 100 000 population, age-standardized)') &\
            (data['period']==2016)]

data['percent'] = data['percent'].astype('float')

datas['udr'] = data[['country','percent']].sort_values(by='percent', ascending=False)

datas['udr']

,country,percent
5763,Tajikistan,129.30
3207,Kyrgyzstan,110.70
2235,Georgia,101.80
6339,Uzbekistan,81.14
6051,Turkmenistan,79.30
4755,Republic of Moldova,78.28
6123,Ukraine,70.72
327,Azerbaijan,63.95
3063,Kazakhstan,62.68
507,Belarus,60.74
